# Make predictions and assess validation metric #
*ToDo: wrap this whole thing in a wandb logger

In [ ]:
local = False
if local:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/'Kaggle Otto Reccommender'/data
  path_to_module = '/content/drive/MyDrive/Kaggle Otto Reccommender/'
else:
  !mkdir /my_mnt_dir
  !google-drive-ocamlfuse /my_mnt_dir
  %cd /my_mnt_dir/'Kaggle Otto Reccommender'/data
  path_to_module = '/my_mnt_dir/Kaggle Otto Reccommender/'

import sys    
sys.path.append(path_to_module)

In [ ]:
!pip install fastparquet
!pip install polars

In [ ]:
import glob
import numpy as np
import polars as pl
import pandas as pd
import gc
from otto_utils import get_train, get_test, make_directory, convert_columns, save_parquet
from tqdm import tqdm

In [ ]:
# constants: ToDo: Move to config
validation = False
path_to_training_data = './train_training_data' if validation else './test_training_data'

In [ ]:
training_skeleton = pd.read_parquet(f'{path_to_training_data}/training_skeleton', engine='fastparquet')

In [ ]:
training_skeleton = pl.from_pandas(training_skeleton)

In [ ]:
if validation:
  test = pd.read_parquet(f'validation/test_parquet', engine='fastparquet')
  test = pl.from_pandas(test)
  train = pd.read_parquet(f'validation/train_parquet', engine='fastparquet')
  train = pl.from_pandas(train)
else:
  test = pd.read_parquet('test/test_parquet', engine='fastparquet')
  test = pl.from_pandas(test)
  train = pd.read_parquet('test/train_parquet', engine='fastparquet')
  train = pl.from_pandas(train)

In [ ]:
train = (train
         .with_column(pl.col('session').cast(pl.Int32).keep_name())
         .with_column(pl.col('aid').cast(pl.Int32).keep_name())
)
test = (test
         .with_column(pl.col('session').cast(pl.Int32).keep_name())
         .with_column(pl.col('aid').cast(pl.Int32).keep_name())
)

We build many different types of features:
* Customer (session) features:
  * Total clicks a customer made
  * Total carts
  * Total orders
  * Unique aids selected
* Aid features (train)
  * Train: Amount of clicks, carts, orders
  * Train: % of clicks that turned to orders
  * Test: Amout of clicks, carts, orders
  * Train & Test: Test clicks / train clicks
* Customer-item sessinos
  * Amount of clicks/carts/orders of each AID on the mode date of transaction. 
  * Todays popularity versus same day last week.
  * Count of clicks/carts/orders in session
  * How recently the item was iteracted with by the custer - most recent 0, second last iteration 1 etc.
  * Actual time in minutes between aid seen and end of transaction.
  * Time between end of transaction and aid last seen in a different transaction

In [ ]:
for action_type in ['clicks', 'carts', 'orders']:
  test = test.with_column((pl.col("type") == action_type).alias(action_type))
  train = train.with_column((pl.col("type") == action_type).alias(action_type))

In [ ]:
min_date = (test
            .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
            .select(pl.col('date')).min()
            .item()
            )

In [ ]:
# Always use 3 weeks for training so train and validation match
# if last 3 weeks:
week_count = len(
    train
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .select(['date'])
    .unique()) / 7
week_count = int(week_count)
print(f'{week_count} weeks in data')

Session Aid Features

In [ ]:
# Session features
session_features = (
    test
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).alias('timestamp'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).apply(lambda x: x.hour).alias('hour'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).apply(lambda x: x.minute + 60*x.hour).alias('minutes_since_midnight'))
    .groupby('session')
    .agg([
        pl.col('clicks').sum().cast(pl.Int16).alias('session_click_count'),
        pl.col('carts').sum().cast(pl.Int16).alias('session_cart_count'),
        pl.col('orders').sum().cast(pl.Int16).alias('session_order_count'),
        pl.col('ts').count().cast(pl.Int16).alias('session_total_interactions'),
        pl.col('aid').n_unique().cast(pl.Int16).alias('session_unique_aids'),
        pl.col('date').min().alias('transaction_start_date'),
        pl.col('hour').min().cast(pl.Int8).alias('transaction_start_hour'),
        pl.col('hour').max().cast(pl.Int8).alias('transaction_end_hour'),
        pl.col('timestamp').min().alias('transaction_start_timestamp'),
        pl.col('timestamp').max().alias('transaction_end_timestamp')
    ])
    .with_column((pl.col('transaction_start_date') - min_date).apply(lambda x: x.days).cast(pl.Int8).alias('days_since_test_began'))
    .with_column((pl.col('transaction_end_timestamp') - pl.col('transaction_start_timestamp')).apply(lambda x: x.total_seconds() / 60).cast(pl.Int16).alias('transaction_length_minutes'))
    .with_column((pl.col('session_total_interactions') / pl.col('transaction_length_minutes')).cast(pl.Float32).alias('actions_per_minute'))
    .drop(['transaction_start_date', 'transaction_end_timestamp', 'transaction_start_timestamp'])
)
# Session total clicks, session total carrts, session total orders
session_features

<ipython-input-14-1485f61c1655>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  test


session,session_click_count,session_cart_count,session_order_count,session_total_interactions,session_unique_aids,transaction_start_hour,transaction_end_hour,days_since_test_began,transaction_length_minutes,actions_per_minute
i32,i16,i16,i16,i16,i16,i8,i8,i8,i16,f32
13060512,7,0,0,7,7,17,22,0,7470,0.000937
13737536,22,0,0,22,20,8,11,3,190,0.115789
13891072,3,0,0,3,1,20,21,3,6,0.5
14050560,1,0,0,1,1,17,17,4,0,inf
13530496,3,0,0,3,3,12,12,2,8,0.375
13541568,13,0,0,13,13,13,13,2,16,0.8125
13718880,2,1,0,3,2,0,0,3,0,inf
13742688,4,0,0,4,4,8,15,3,382,0.010471
13782752,1,0,0,1,1,12,12,3,0,inf


In [ ]:
# Do the sessions involve popular items? 
test_item_popularity = (
  test
  .groupby('aid')
  .agg([pl.col('ts').count().cast(pl.Int32).alias('test_aid_popularity')]) 
)

train_item_popularity = (
    train
    .groupby('aid')
    .agg([(pl.col('ts').count() / week_count).cast(pl.Float32).alias('train_aid_popularity')])
)

sessions_with_aid_popularity = (
    test
    .join(
        test_item_popularity,
        on='aid',
        how='left')
    .join(
        train_item_popularity,
        on='aid',
        how='left')
    .groupby('session')
    .agg([
        pl.col('test_aid_popularity').mean().cast(pl.Int32).alias('session_mean_aid_test_popularity'),
        pl.col('train_aid_popularity').mean().cast(pl.Float32).alias('session_mean_aid_train_popularity')])
)

session_features = (
    session_features.join(
        sessions_with_aid_popularity,
        on='session',
        how='left'
    )
)
session_features

session,session_click_count,session_cart_count,session_order_count,session_total_interactions,session_unique_aids,transaction_start_hour,transaction_end_hour,days_since_test_began,transaction_length_minutes,actions_per_minute,session_mean_aid_test_popularity,session_mean_aid_train_popularity
i32,i16,i16,i16,i16,i16,i8,i8,i8,i16,f32,i32,f32
13060512,7,0,0,7,7,17,22,0,7470,0.000937,62,664.214294
13737536,22,0,0,22,20,8,11,3,190,0.115789,33,149.806824
13891072,3,0,0,3,1,20,21,3,6,0.5,130,1744.0
14050560,1,0,0,1,1,17,17,4,0,inf,8,31.25
13530496,3,0,0,3,3,12,12,2,8,0.375,105,967.666687
13541568,13,0,0,13,13,13,13,2,16,0.8125,273,605.961548
13718880,2,1,0,3,2,0,0,3,0,inf,17,163.416672
13742688,4,0,0,4,4,8,15,3,382,0.010471,292,2499.25
13782752,1,0,0,1,1,12,12,3,0,inf,1,7.25


Aid Test Aid Features

In [ ]:
# Test aid features
test_aid_features = (
    test
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .with_column(pl.col('ts').cumcount().over(pl.col('session')).alias('interaction_order_ascending'))
    .with_column(pl.col('session').n_unique().over(pl.lit(1)).alias('total_sessions'))
    .groupby('aid')
    .agg([
        pl.col('clicks').sum().cast(pl.Int32).alias('test_aid_click_count'),
        pl.col('carts').sum().cast(pl.Int32).alias('test_aid_cart_count'),
        pl.col('orders').sum().cast(pl.Int32).alias('test_aid_order_count'),
        pl.col('session').n_unique().cast(pl.Int32).alias('test_aid_unique_sessions'),
        (pl.col('date').n_unique() / (7*week_count)).cast(pl.Float32).alias('percent_of_test_weeks_interacted'),
        (100 * pl.col('session').n_unique() / pl.col('total_sessions').max()).cast(pl.Float32).alias('test_session_penetration_percent'),
        pl.col('ts').count().cast(pl.Int32).alias('test_aid_total_interactions'),
        
        pl.when(pl.col('interaction_order_ascending') == 0).then(1).otherwise(0).sum().cast(pl.Int32).alias('test_aid_first_interaction_count')
    ])
)
test_aid_features

<ipython-input-16-2afe76f239db>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  test


aid,test_aid_click_count,test_aid_cart_count,test_aid_order_count,test_aid_unique_sessions,percent_of_test_weeks_interacted,test_session_penetration_percent,test_aid_total_interactions,test_aid_first_interaction_count
i32,i32,i32,i32,i32,f32,f32,i32,i32
1070336,32,3,0,21,0.25,0.001256,35,2
1176480,1,0,0,1,0.035714,0.00006,1,1
1512352,1,0,0,1,0.035714,0.00006,1,0
1363840,1,0,0,1,0.035714,0.00006,1,0
5952,2,0,0,2,0.071429,0.00012,2,0
1127744,2,0,0,2,0.071429,0.00012,2,0
261408,3,0,0,3,0.071429,0.000179,3,0
205152,2,0,0,2,0.071429,0.00012,2,0
36544,16,0,0,6,0.142857,0.000359,16,0


Aid Train Features averaged by week

In [ ]:
# test vs train trend:
train_aid_features = (
    train
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .with_column(pl.col('ts').cumcount().over(pl.col('session')).alias('interaction_order_ascending'))
    .groupby(['aid'])
    .agg([
        (pl.col('clicks').sum() / week_count).cast(pl.Float32).alias('train_aid_click_sum'),
        (pl.col('carts').sum() / week_count).cast(pl.Float32).alias('train_aid_cart_sum'),
        (pl.col('orders').sum() / week_count).cast(pl.Float32).alias('train_aid_order_sum'),
        (pl.col('session').n_unique() / week_count).cast(pl.Float32).alias('train_aid_session_distinct_count'),
        (pl.col('date').n_unique() / (7*week_count)).cast(pl.Float32).alias('percent_of_train_weeks_interacted'),
        (pl.col('ts').count() / week_count).cast(pl.Float32).alias('train_aid_total_interactions'),
        (pl.when(pl.col('interaction_order_ascending') == 0).then(1).otherwise(0).sum() / week_count).cast(pl.Float32).alias('train_aid_first_interaction_count')
    ])
    .with_column((pl.col('train_aid_order_sum') / pl.col('train_aid_cart_sum')).cast(pl.Float32).alias('order_to_carts_ratio'))
    .with_column((pl.col('train_aid_total_interactions') / pl.col('train_aid_session_distinct_count')).cast(pl.Float32).alias('interactions_per_session'))
    .with_column((pl.col('train_aid_click_sum') / pl.col('train_aid_session_distinct_count')).cast(pl.Float32).alias('clicks_per_session'))
    .with_column(((pl.col('train_aid_order_sum')+pl.col('train_aid_cart_sum')) / pl.col('train_aid_session_distinct_count')).cast(pl.Float32).alias('carts_and_orders_per_session'))
    .with_column((pl.col('train_aid_cart_sum') / pl.col('train_aid_click_sum')).cast(pl.Float32).alias('carts_to_clicks_ratio'))
)
train_aid_features

<ipython-input-19-4998b85c8769>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  train


aid,train_aid_click_sum,train_aid_cart_sum,train_aid_order_sum,train_aid_session_distinct_count,percent_of_train_weeks_interacted,train_aid_total_interactions,train_aid_first_interaction_count,order_to_carts_ratio,interactions_per_session,clicks_per_session,carts_and_orders_per_session,carts_to_clicks_ratio
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
31040,2.25,0.25,0.0,1.5,0.25,2.5,0.75,0.0,1.666667,1.5,0.166667,0.111111
513536,1.25,0.0,0.0,0.5,0.071429,1.25,0.0,NaN,2.5,2.5,0.0,0.0
1848800,4.75,0.0,0.0,4.5,0.5,4.75,0.25,NaN,1.055556,1.055556,0.0,0.0
788768,1.5,1.25,0.25,0.5,0.142857,3.0,0.0,0.2,6.0,3.0,3.0,0.833333
534848,2.5,0.0,0.0,2.0,0.321429,2.5,0.0,NaN,1.25,1.25,0.0,0.0
1655360,273.0,23.0,8.25,192.25,1.0,304.25,2.75,0.358696,1.582575,1.420026,0.162549,0.084249
1711456,3.25,0.25,0.0,2.25,0.25,3.5,0.0,0.0,1.555556,1.444444,0.111111,0.076923
457664,7.25,0.25,0.25,5.25,0.535714,7.75,1.0,1.0,1.47619,1.380952,0.095238,0.034483
240544,7.75,1.0,0.25,5.5,0.571429,9.0,1.0,0.25,1.636364,1.409091,0.227273,0.129032


In [ ]:
# reorder amount:
train_reorder = (
    train
    .filter(pl.col('orders') == 1)
    .groupby(['aid', 'session'])
    .agg(pl.col('ts').count().alias('count_of_orders'))
    .groupby('aid')
    .agg([
        (pl.when(pl.col('count_of_orders') > 1).then(1).otherwise(0).sum() / week_count).cast(pl.Float32).alias('train_distinct_sessions_that_reorder'),
        (pl.col('session').count() / week_count).cast(pl.Float32).alias('train_aid_distinct_orders')])
)

In [ ]:
train_aid_features = (
    train_aid_features
    .join(
        train_reorder,
        how='left',
        on='aid')
    .with_column((pl.col('train_distinct_sessions_that_reorder') / pl.col('train_aid_distinct_orders')).cast(pl.Float32).alias('train_aid_reorder_rate'))
)
train_aid_features.filter(pl.col('train_distinct_sessions_that_reorder') > 1)

<ipython-input-21-7283ec729d79>:2: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  train_aid_features


aid,train_aid_click_sum,train_aid_cart_sum,train_aid_order_sum,train_aid_session_distinct_count,percent_of_train_weeks_interacted,train_aid_total_interactions,train_aid_first_interaction_count,order_to_carts_ratio,interactions_per_session,clicks_per_session,carts_and_orders_per_session,carts_to_clicks_ratio,train_distinct_sessions_that_reorder,train_aid_distinct_orders,train_aid_reorder_rate
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
369920,864.0,173.25,45.75,508.5,1.0,1083.0,39.75,0.264069,2.129793,1.699115,0.430678,0.200521,2.0,43.75,0.045714
1052000,312.0,64.25,33.25,257.5,1.0,409.5,9.5,0.51751,1.590291,1.21165,0.378641,0.205929,1.5,31.75,0.047244
506144,179.25,38.0,21.25,133.0,1.0,238.5,10.0,0.559211,1.793233,1.347744,0.445489,0.211994,1.25,19.75,0.063291
827776,450.5,113.5,51.25,307.0,1.0,615.25,15.0,0.451542,2.004072,1.467427,0.536645,0.251942,7.5,43.5,0.172414
1647200,61.5,13.5,7.25,39.75,0.928571,82.25,0.5,0.537037,2.069182,1.54717,0.522013,0.219512,1.5,5.5,0.272727
899360,143.75,25.25,12.25,104.5,0.928571,181.25,7.25,0.485149,1.73445,1.375598,0.358852,0.175652,1.75,9.75,0.179487
518592,151.75,22.75,10.75,107.75,1.0,185.25,3.5,0.472527,1.719258,1.408353,0.310905,0.149918,1.25,9.0,0.138889
438208,179.0,34.0,12.75,145.25,1.0,225.75,17.0,0.375,1.554217,1.232358,0.321859,0.189944,1.75,10.75,0.162791
1620640,201.75,72.0,30.0,153.75,1.0,303.75,8.75,0.416667,1.97561,1.312195,0.663415,0.356877,3.5,26.0,0.134615


In [ ]:
# Calculate the proportion of item sales in morning, evening and night. 
periodic_sales = (
    train
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).apply(lambda x: x.hour).alias('hour'))
    .with_column(pl.when((pl.col('hour') <= 4) | (pl.col('hour') >= 20)).then(pl.lit(1)).otherwise(pl.lit(0)).alias('night_interactions'))
    .with_column(pl.when((pl.col('hour') >= 5) & (pl.col('hour') <= 12)).then(pl.lit(1)).otherwise(pl.lit(0)).alias('morning_interactions'))
    .with_column(pl.when((pl.col('hour') >= 13) & (pl.col('hour') <= 19)).then(pl.lit(1)).otherwise(pl.lit(0)).alias('afternoon_interactions'))
    .groupby('aid')
    .agg([
        (pl.col('night_interactions').sum() / pl.col('ts').count()).cast(pl.Float32).alias('night_prop'),
        (pl.col('morning_interactions').sum() / pl.col('ts').count()).cast(pl.Float32).alias('morning_prop'),
        (pl.col('afternoon_interactions').sum() / pl.col('ts').count()).cast(pl.Float32).alias('afternoon_prop'),
    ])
)

train_aid_features = (
    train_aid_features
    .join(periodic_sales,
          on='aid',
          how='left')
)

train_aid_features

<ipython-input-22-652d6952410e>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  train


aid,train_aid_click_sum,train_aid_cart_sum,train_aid_order_sum,train_aid_session_distinct_count,percent_of_train_weeks_interacted,train_aid_total_interactions,train_aid_first_interaction_count,order_to_carts_ratio,interactions_per_session,clicks_per_session,carts_and_orders_per_session,carts_to_clicks_ratio,train_distinct_sessions_that_reorder,train_aid_distinct_orders,train_aid_reorder_rate,night_prop,morning_prop,afternoon_prop
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
31040,2.25,0.25,0.0,1.5,0.25,2.5,0.75,0.0,1.666667,1.5,0.166667,0.111111,null,null,null,0.2,0.2,0.6
513536,1.25,0.0,0.0,0.5,0.071429,1.25,0.0,NaN,2.5,2.5,0.0,0.0,null,null,null,0.0,0.2,0.8
1848800,4.75,0.0,0.0,4.5,0.5,4.75,0.25,NaN,1.055556,1.055556,0.0,0.0,null,null,null,0.526316,0.052632,0.421053
788768,1.5,1.25,0.25,0.5,0.142857,3.0,0.0,0.2,6.0,3.0,3.0,0.833333,0.0,0.25,0.0,0.25,0.0,0.75
534848,2.5,0.0,0.0,2.0,0.321429,2.5,0.0,NaN,1.25,1.25,0.0,0.0,null,null,null,0.4,0.2,0.4
1655360,273.0,23.0,8.25,192.25,1.0,304.25,2.75,0.358696,1.582575,1.420026,0.162549,0.084249,1.0,7.25,0.137931,0.395234,0.251438,0.353328
1711456,3.25,0.25,0.0,2.25,0.25,3.5,0.0,0.0,1.555556,1.444444,0.111111,0.076923,null,null,null,0.428571,0.142857,0.428571
457664,7.25,0.25,0.25,5.25,0.535714,7.75,1.0,1.0,1.47619,1.380952,0.095238,0.034483,0.0,0.25,0.0,0.193548,0.225806,0.580645
240544,7.75,1.0,0.25,5.5,0.571429,9.0,1.0,0.25,1.636364,1.409091,0.227273,0.129032,0.0,0.25,0.0,0.222222,0.583333,0.194444


Aid Trend Features - Day on Day

In [ ]:
# Calculate todays interactions, last week this days interactions, and todays uplift. 
date_sums = (
    test
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .with_column((pl.col('date') - 7).cast(pl.Date).alias('last_week'))
    .groupby(['date','last_week','aid'])
    .agg(pl.col('ts').count().alias('interactions'))
)

last_week_sums = (
    train
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .groupby(['date', 'aid'])
    .agg(pl.col('ts').count().alias('interactions'))
)

day_on_day_trend = (
    date_sums
    .join(
      last_week_sums,
      left_on=['last_week', 'aid'],
      right_on=['date','aid'],
      how='left')
    .with_column(
        (pl.col('interactions') / pl.col('interactions_right'))
        .cast(pl.Float32)
        .alias('test_aid_daily_uplift'))
        .rename({'interactions' : 'test_aid_todays_interactions', 'interactions_right' : 'test_aid_last_week_todays_interactions'})
        .select(['date', 'aid', 'test_aid_todays_interactions', 'test_aid_last_week_todays_interactions', 'test_aid_daily_uplift'])
)
day_on_day_trend.limit(15)
# We get todays orders, this time last weeks orders and uplift. 

<ipython-input-23-1717c2114f5a>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  test
<ipython-input-23-1717c2114f5a>:11: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  train
<ipython-input-23-1717c2114f5a>:18: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  date_sums


date,aid,test_aid_todays_interactions,test_aid_last_week_todays_interactions,test_aid_daily_uplift
date,i32,u32,u32,f32
2022-08-29,767749,2,29,0.068966
2022-09-01,1741269,1,12,0.083333
2022-08-31,757885,1,8,0.125
2022-09-03,1304302,2,12,0.166667
2022-09-02,1151843,2,8,0.25
2022-09-02,1162594,1,8,0.125
2022-08-30,1341750,1,1,1.0
2022-09-03,1854469,13,53,0.245283
2022-08-30,1188990,1,2,0.5


For each session, calculate the time since the candidate has been clicked and ordered. E.g you see the item at 8pm, and it was purchased 60 minutes ago at 7pm

In [ ]:
# We need a mapping of session to date:
session_to_date = (
    test
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .groupby(pl.col('session'))
    .agg([
        pl.col('date').max().alias('date'),
        pl.col('ts').max().alias('ts')])
)
session_to_date

In [ ]:
orders = (
    pl.concat([
        (train
            .filter(pl.col('orders') == 1)
            .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
            .select(['session','aid','ts'])),
        test.filter(pl.col('orders') == 1).select(['session','aid','ts'])])
    .with_column(pl.col('ts').alias('ts_right'))
    .sort('ts')
)

clicks = (
    pl.concat([
        (train
            .filter(pl.col('clicks') == 1)
            .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
            .select(['session','aid','ts'])),
        test.filter(pl.col('clicks') == 1).select(['session','aid','ts'])])
    .with_column(pl.col('ts').alias('ts_right'))
    .sort('ts')
)

In [ ]:
last_ordered = (
    training_skeleton.select(['session', 'aid'])
    .join(
        session_to_date.select(['session','ts']),
        on='session',
        how='left'
        )
    .sort('ts')
    .join_asof(
        orders,
        by='aid',
        on='ts',
        strategy = 'backward'
    )
    .filter(pl.col('session') != pl.col('session_right'))
    .with_column(((pl.col('ts') - pl.col('ts_right')) / (1000*60)).cast(pl.Float32).alias('minutes_since_last_order'))
    .select(['session', 'aid','minutes_since_last_order'])
)

last_clicked = (
    training_skeleton.select(['session', 'aid'])
    .join(
        session_to_date.select(['session','ts']),
        on='session',
        how='left'
        )
    .sort('ts')
    .join_asof(
        clicks,
        by='aid',
        on='ts',
        strategy = 'backward'
    )
    .filter(pl.col('session') != pl.col('session_right'))
    .with_column(((pl.col('ts') - pl.col('ts_right')) / (1000*60)).cast(pl.Float32).alias('minutes_since_last_click'))
    .select(['session', 'aid','minutes_since_last_click'])
)

last_actions = (
    last_ordered
    .join(
        last_clicked,
        on=['session','aid'],
        how='left'
    )
)
last_actions

<ipython-input-26-5021dc97cff4>:2: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  training_skeleton.select(['session', 'aid'])
<ipython-input-26-5021dc97cff4>:21: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  training_skeleton.select(['session', 'aid'])


session,aid,minutes_since_last_order,minutes_since_last_click
i32,i32,f32,f32
12899779,59625,null,null
12899779,1253524,null,135.63855
12899779,737445,null,1.392367
12899779,438191,3597.470703,48.251633
12899779,731692,3243.108887,6.924117
12899779,1790770,35257.0625,1927.602905
12899779,94230,15957.460938,7.760117
12899779,1660529,null,118.283081
12899779,199422,3782.551514,56.530968


Session - Aid Features

In [ ]:
session_aid_features = (
    test
    .with_column(pl.col('ts').cumcount().over(pl.col('session')).alias('interaction_order_ascending'))
    .with_column(pl.col('ts').cumcount(reverse=True).over(pl.col('session')).alias('interaction_order_descending'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).alias('timestamp'))
    .with_column(pl.col('timestamp').max().over('session').alias('end_timestamp'))
    .with_column((pl.col('end_timestamp') - pl.col('timestamp')).apply(lambda x: x.total_seconds() / 60).cast(pl.Float32).alias('session_aid_minutes_since_interaction'))
    .with_column((pl.col('ts')*1000 + 2*60*60*1000*1000).cast(pl.Datetime).cast(pl.Date).alias('date'))
    .groupby(['aid','session'])
    .agg([
        pl.col('clicks').sum().cast(pl.Int32).alias('session_aid_click_count'),
        pl.col('carts').sum().cast(pl.Int32).alias('session_aid_cart_count'),
        pl.col('orders').sum().cast(pl.Int32).alias('session_aid_order_count'),
        pl.col('interaction_order_ascending').min().cast(pl.Int32).alias('session_aid_clicks_since_start'),
        pl.col('interaction_order_descending').min().cast(pl.Int32).alias('session_aid_clicks_since_end'),
        pl.col('session_aid_minutes_since_interaction').min().cast(pl.Float32).alias('session_aid_minutes_since_interaction'),
        pl.col('ts').count().cast(pl.Int32).alias('session_aid_total_interactions')
    ])
)
session_aid_features

<ipython-input-27-ee0d7296ba2d>:2: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  test


aid,session,session_aid_click_count,session_aid_cart_count,session_aid_order_count,session_aid_clicks_since_start,session_aid_clicks_since_end,session_aid_minutes_since_interaction,session_aid_total_interactions
i32,i32,i32,i32,i32,i32,i32,f32,i32
795200,13826317,1,0,0,2,6,11.910017,1
307904,14429869,1,0,0,14,19,479.467194,1
763424,14014474,1,0,0,9,6,77.67907,1
1196256,14256006,2,0,0,4,0,0.0,2
706784,14258713,2,0,0,10,15,152.930649,2
963968,13065360,1,0,0,1,1,0.489967,1
829376,13726212,1,0,0,5,0,0.0,1
167744,12967941,1,0,0,3,11,12.857767,1
811616,13934840,1,0,0,1,4,4.25945,1


## Merge on the features to the candidate list (training skeleton) ##

In [ ]:
training_skeleton = training_skeleton.join(
    session_features,
    how='left',
    on='session'
)

In [ ]:
training_skeleton = training_skeleton.join(
    test_aid_features,
    how='left',
    on='aid'
)

In [ ]:
training_skeleton = training_skeleton.join(
    train_aid_features,
    how='left',
    on='aid'
)

In [ ]:
training_skeleton = (
    training_skeleton
    .join(
        session_to_date.select(['session','date']),
        how='left',
        on='session')
    .join(
        day_on_day_trend,
        on=['date','aid'],
        how='left'
    )
    .drop('date')
)

In [ ]:
training_skeleton = (
    training_skeleton
    .join(
        last_actions,
        how='left',
        on=['session','aid']
    )
)

In [ ]:
training_skeleton = training_skeleton.join(
    session_aid_features,
    how='left',
    on=['session', 'aid']
)
training_skeleton

session,aid,weight,n_basket,also_buy_pairings,n_also_buy,pairings,n_covisitation,cart_order_pairings,n_cart_order,w2vec_dist,n_word2vec,confidence,n_als,session_click_count,session_cart_count,session_order_count,session_total_interactions,session_unique_aids,transaction_start_hour,transaction_end_hour,days_since_test_began,transaction_length_minutes,actions_per_minute,session_mean_aid_test_popularity,session_mean_aid_train_popularity,test_aid_click_count,test_aid_cart_count,test_aid_order_count,test_aid_unique_sessions,percent_of_test_weeks_interacted,test_session_penetration_percent,test_aid_total_interactions,test_aid_first_interaction_count,train_aid_click_sum,train_aid_cart_sum,train_aid_order_sum,train_aid_session_distinct_count,percent_of_train_weeks_interacted,train_aid_total_interactions,train_aid_first_interaction_count,order_to_carts_ratio,interactions_per_session,clicks_per_session,carts_and_orders_per_session,carts_to_clicks_ratio,train_distinct_sessions_that_reorder,train_aid_distinct_orders,train_aid_reorder_rate,night_prop,morning_prop,afternoon_prop,test_aid_todays_interactions,test_aid_last_week_todays_interactions,test_aid_daily_uplift,minutes_since_last_order,minutes_since_last_click,session_aid_click_count,session_aid_cart_count,session_aid_order_count,session_aid_clicks_since_start,session_aid_clicks_since_end,session_aid_minutes_since_interaction,session_aid_total_interactions
i32,i32,f32,i16,f32,i16,f32,i16,f32,i16,f32,i16,f32,i16,i16,i16,i16,i16,i16,i8,i8,i8,i16,f32,i32,f32,i32,i32,i32,i32,f32,f32,i32,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u32,u32,f32,f32,f32,i32,i32,i32,i32,i32,f32,i32
12899779,59625,0.071773,1,null,null,null,null,null,null,null,null,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,1,0,0,1,0.035714,0.00006,1,1,3.0,0.0,0.0,2.75,0.357143,3.0,0.25,NaN,1.090909,1.090909,0.0,0.0,null,null,null,0.583333,0.083333,0.333333,1,null,null,null,null,1,0,0,0,0,0.0,1
12899779,1253524,null,null,null,null,6.42786,1,null,null,null,null,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,4,0,0,4,0.107143,0.000239,4,2,67.25,1.5,0.0,52.0,1.0,68.75,1.5,0.0,1.322115,1.293269,0.028846,0.022305,null,null,null,0.363636,0.203636,0.432727,null,null,null,null,135.63855,null,null,null,null,null,null,null
12899779,737445,null,null,null,null,4.967372,2,4.0,11,null,null,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,34,1,0,25,0.25,0.001495,35,15,227.25,3.75,0.0,143.75,1.0,231.0,16.75,0.0,1.606956,1.58087,0.026087,0.016502,null,null,null,0.33658,0.237013,0.426407,4,30,0.133333,null,1.392367,null,null,null,null,null,null,null
12899779,438191,null,null,null,null,4.209438,3,null,null,null,null,0.000029,47,1,0,0,1,1,0,0,0,0,inf,1,3.0,59,2,0,45,0.25,0.002692,61,17,868.5,28.0,1.0,565.0,1.0,897.5,54.25,0.035714,1.588496,1.537168,0.051327,0.032239,0.0,1.0,0.0,0.359053,0.239554,0.401393,6,59,0.101695,3597.470703,48.251633,null,null,null,null,null,null,null
12899779,731692,null,null,null,null,4.123152,4,4.0,10,null,null,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,5,2,0,4,0.107143,0.000239,7,0,26.5,3.75,1.25,14.25,0.857143,31.5,0.75,0.333333,2.210526,1.859649,0.350877,0.141509,0.25,1.0,0.25,0.269841,0.238095,0.492063,null,null,null,3243.108887,6.924117,null,null,null,null,null,null,null
12899779,1790770,null,null,null,null,3.518961,5,null,null,null,null,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,null,null,null,null,null,null,null,null,8.25,0.5,0.25,6.5,0.678571,9.0,1.0,0.5,1.384615,1.269231,0.115385,0.060606,0.0,0.25,0.0,0.166667,0.555556,0.277778,null,null,null,35257.0625,1927.602905,null,null,null,null,null,null,null
12899779,94230,null,null,null,null,3.293997,6,null,null,0.498162,72,null,null,1,0,0,1,1,0,0,0,0,inf,1,3.0,25,1,0,20,0.178571,0.001196,26,12,133.25,3.25,0.25,92.25,1.0,136.75,13.25,0.076923,1.482385,1.444444,0.03794,0.02439,0.0,0.25,0.0,0.274223,0.221207,0.50457,6,40,0.15,15957.460938,7.760117,null,null,null,null,null,null,null
12899779,1660529,null,null,null,null,3.293997,7,null,null,null,null,null,null,1,0,0,1,1,0,0,0,0,i

In [ ]:
# Make some last columns:
training_skeleton = (
    training_skeleton
    .with_column((pl.col('test_aid_total_interactions') / pl.col('session_mean_aid_test_popularity')).cast(pl.Float32).alias('test_aid_interactions_to_test_popularity'))
    .with_column((pl.col('train_aid_total_interactions') / pl.col('session_mean_aid_train_popularity')).cast(pl.Float32).alias('train_aid_interactions_to_train_popularity'))
)

<ipython-input-38-3f4b4510fc0f>:3: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  training_skeleton


In [ ]:
if not validation:
  rows = training_skeleton.shape[0] / 100
  training_skeleton.write_parquet(f'{path_to_training_data}/training_data.parquet' ,
                                  compression='snappy',
                                  row_group_size=rows
                                  )

## If we're building the validation data we also need to attach the labels so we can train models ## 

In [ ]:
tar = pd.read_parquet('./validation/test_labels.parquet')
tar = tar.loc[tar['type']=='clicks']
tar = tar.explode('ground_truth')
tar.rename(columns={'ground_truth' : 'aid'}, inplace=True)
tar.drop(columns='type', inplace=True)
tar['click_response'] = 1
tar = pl.from_pandas(tar)
tar = tar.with_column(pl.col('aid').cast(pl.Int32).alias('aid'))
tar = tar.with_column(pl.col('session').cast(pl.Int32).alias('session'))
training_skeleton = training_skeleton.join(
    tar,
    how='left',
    on=['session','aid']
)
training_skeleton = training_skeleton.with_column(pl.col('click_response').fill_null(0).cast(pl.Int8).alias('click_response'))

In [ ]:
tar = pd.read_parquet('./validation/test_labels.parquet')
tar = tar.loc[tar['type']=='carts']
tar = tar.explode('ground_truth')
tar.rename(columns={'ground_truth' : 'aid'}, inplace=True)
tar.drop(columns='type', inplace=True)
tar['cart_response'] = 1
tar = pl.from_pandas(tar)
tar = tar.with_column(pl.col('aid').cast(pl.Int32).alias('aid'))
tar = tar.with_column(pl.col('session').cast(pl.Int32).alias('session'))
training_skeleton = training_skeleton.join(
    tar,
    on=['session', 'aid'],
    how='left',
)
training_skeleton = training_skeleton.with_column(pl.col('cart_response').fill_null(0).cast(pl.Int8).alias('cart_response'))
training_skeleton.shape

In [ ]:
tar = pd.read_parquet('./validation/test_labels.parquet')
tar = tar.loc[tar['type']=='orders']
tar = tar.explode('ground_truth')
tar.rename(columns={'ground_truth' : 'aid'}, inplace=True)
tar.drop(columns='type', inplace=True)
tar['order_response'] = 1
tar = pl.from_pandas(tar)
tar = tar.with_column(pl.col('aid').cast(pl.Int32).alias('aid'))
tar = tar.with_column(pl.col('session').cast(pl.Int32).alias('session'))

training_skeleton = training_skeleton.join(
    tar,
    on=['session', 'aid'],
    how='left',
)
training_skeleton = training_skeleton.with_column(pl.col('order_response').fill_null(0).cast(pl.Int8).alias('order_response'))

In [ ]:
rows = training_skeleton.shape[0] / 100
training_skeleton.write_parquet(f'{path_to_training_data}/training_data_2.parquet' ,
                                compression='snappy',
                                row_group_size=rows
                                )